In [1]:
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import Search
from functools import reduce
import math

# No need since 28Apr23 - Lat/Lon provided by Airbus
# Convert BNG to LAT/LON
#from bng_latlon import OSGB36toWGS84
#
input_coord_csv_file='data/vichara/output/vichara_loan_flood_data.csv/part-00000-3a44833d-b5a8-45c9-962c-87bc49bca8ec-c000.csv'
df_vichara_loanbook = pd.read_csv(input_coord_csv_file, header = 0 )

/home/notebookuser/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (19,21,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_vichara_loanbook.dtypes

id                                        object
loan_id                                   object
latitude                                 float64
longitude                                float64
river_floodscore_def                       int64
river_floodscore_ud                        int64
coastal_floodscore_ud                      int64
surfacewater_floodscore_ud                 int64
floodscore_def                             int64
floodscore_ud                              int64
river_floodscore_def_rcp45_2050            int64
river_floodscore_ud_rcp45_2050             int64
surfacewater_floodscore_ud_rcp45_2050      int64
river_floodscore_def_rcp85_2050            int64
river_floodscore_ud_rcp85_2050             int64
surfacewater_floodscore_ud_rcp85_2050      int64
asofmon                                   object
servicer_name                             object
loan_age                                   int64
rem_months_legal_maturity                 object
adj_rem_months_matur

In [3]:
# Increase display in Jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
df_vichara_loanbook.shape

(117829, 102)

In [5]:
# partition the dataset as per property_zip
property_zip = df_vichara_loanbook.property_zip.unique()
dict_vichara_df = {}
for zip in property_zip:
    df = df_vichara_loanbook.loc[df_vichara_loanbook['property_zip'] == zip].copy()
    df.reset_index(drop=True, inplace=True)
    dict_vichara_df[str(zip)] = df

In [6]:
#check for partition and size
for key in dict_vichara_df:
    for d in dict_vichara_df[key].iterrows():
        pass
    print(key, d[0])

335 33726
346 29970
336 24692
337 29437


In [7]:
lat= 27.994402
lon= -81.760254

In [8]:
#Color codes for flood rating scale
colours_flood={
    'no-colour':'#deebf6',
    'green':'#a5d45d',
    'amber':'#ecc63a',
    'red':'#d92424',
    'black-1':'#919191',
    'black-2':'#242424',
    'purple':'#9370db',
}

# Function to convert number into string
# Switcher is dictionary data type here
### Planned for FloodScore_Def/FloodScore_UD -  Combined Floodability defended/undefended - score
def FloodRatingSwitch(undefend_sum,defend_sum):
    value_search = undefend_sum if (undefend_sum > defend_sum) else defend_sum
    # print('value_search_='+str(value_search))
    list_no_colour = [0]
    list_green = [1,2,3,4,5,6,7,8,9,10]
    list_amber = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    list_red = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]
    list_black1 = [45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
    list_black2 = [61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]
    # Evaluate from List
    if value_search in list_no_colour:
        return 'no-colour'
    elif value_search in list_green:
        return 'green'
    elif value_search in list_amber:
        return 'amber'
    elif value_search in list_red:
        return 'red'
    elif value_search in list_black1:
        return 'black-1'
    elif value_search in list_black2:
        return 'black-2'
    else:
        return 'purple' #ERROR out-of-bounds
    
    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    # return switcher.get(value_search, 'purple') # 'purple':'#9370db', ERROR
    
## Function to Define Ratio Value between 0 ( 0.001) and 100 as %
def FloodRating_Ration_0_100_Value(undefend_sum,defend_sum):
    value_search = undefend_sum if (undefend_sum > defend_sum) else defend_sum
    # augment in 40% for colour grading on map
    ratio_0_100 = (value_search/100)*1.40 + 0.001
    ratio_0_100 = round( ratio_0_100, 4)
    return ratio_0_100

In [9]:
#CO - Condos , MH - manufactured home, PU - , SF - single family residence
df_vichara_loanbook_play = df_vichara_loanbook.groupby('property_type').property_type.count()
df_vichara_loanbook_play

property_type
CO    11094
MH      552
PU    54851
SF    51332
Name: property_type, dtype: int64

In [10]:
#I - Investment properties , P - Primary Residences, S-  Second Homes
df_vichara_loanbook_play = df_vichara_loanbook.groupby('occupancy_status').occupancy_status.count()
df_vichara_loanbook_play

occupancy_status
I      8332
P    101304
S      8193
Name: occupancy_status, dtype: int64

In [11]:
df_vichara_loanbook_play = df_vichara_loanbook.groupby('number_of_units').number_of_units.count()
df_vichara_loanbook_play

number_of_units
1    116864
2       651
3       175
4       139
Name: number_of_units, dtype: int64

In [12]:
# Creating the popup labels for high level flood rating data

def popup_html_flood(row):
    i = row
    property_id=df_vichara_loanbook['id'].iloc[i] 
    address= 'State='+str(df_vichara_loanbook['property_state'].iloc[i])+'. Zip='+str(df_vichara_loanbook['property_zip'].iloc[i])+'. Property type='+str(df_vichara_loanbook['property_type'].iloc[i])+'. Number of units='+str(df_vichara_loanbook['number_of_units'].iloc[i])+'. Occupancy status='+str(df_vichara_loanbook['occupancy_status'].iloc[i]) +'. Latitude/Longitude='+ str( df_vichara_loanbook['latitude'].iloc[i] ) +'/'+ str( df_vichara_loanbook['longitude'].iloc[i] )
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">ID:{}</h4>""".format(property_id) + """</center>
<center> <table style="height: 130px; width: 500px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Floodability score</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Floodability Type </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Defended score </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Undefended score </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_vichara_loanbook['river_floodscore_def'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_vichara_loanbook['river_floodscore_ud'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">-</td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_vichara_loanbook['coastal_floodscore_ud'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">-</td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_vichara_loanbook['surfacewater_floodscore_ud'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Overall score</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df_vichara_loanbook['floodscore_def'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df_vichara_loanbook['floodscore_ud'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [13]:
# Creating the popup labels for detail flood rating data and loan data

def popup_html_flood_loan(row, df):
    i = row
    property_id=df['id'].iloc[i] 
    address= 'State='+str(df['property_state'].iloc[i])+'. Zip='+str(df['property_zip'].iloc[i])+'. Property type='+str(df['property_type'].iloc[i])+'. Number of units='+str(df['number_of_units'].iloc[i])+'. Occupancy status='+str(df['occupancy_status'].iloc[i]) +'. Latitude/Longitude='+ str( df['latitude'].iloc[i] ) +'/'+ str( df['longitude'].iloc[i] )
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">ID:{}</h4>""".format(property_id) + """</center>
<center> <table style="height: 130px; width: 500px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Floodability score</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Floodability Type </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Defended score </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Undefended score </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['river_floodscore_def'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['river_floodscore_ud'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">-</td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['coastal_floodscore_ud'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">-</td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['surfacewater_floodscore_ud'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Overall score</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df['floodscore_def'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df['floodscore_ud'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">JBA Flood Score (0-20) for the RCP 4.5 Climate Change scenario in 2050</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Floodability Type </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Defended score </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Undefended score </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['river_floodscore_def_rcp45_2050'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['river_floodscore_ud_rcp45_2050'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">-</td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['surfacewater_floodscore_ud_rcp45_2050'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">JBA Flood Score (0-20) for the RCP 8.5 Climate Change scenario in 2050</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Floodability Type </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Defended score </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Undefended score </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['river_floodscore_def_rcp85_2050'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['river_floodscore_ud_rcp85_2050'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">-</td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df['surfacewater_floodscore_ud_rcp85_2050'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 130px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Loan ID</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Product Type</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Original Term</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Original Rate</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">First Pay Date</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Maturity Date</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Original UPB</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Original LTV</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current UPB</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Zero Bal Date</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['loan_id'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['product_type'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['orig_loan_term'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}%</td>""".format(df['orig_int_rate'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['first_pay_date'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['maturity_date'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}$</td>""".format(df['orig_upb'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['orig_ltv'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}$</td>""".format(df['cur_actual_upb'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df['zero_bal_date'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    
    #check for orig_ltv if formula is needed or not and check how to add the currency symbol and if percent sign is needed or not 
    return html

In [14]:
#Creating lagends

def add_legend(maps, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_label = {}
    for i in range(len(labels)):
        color_label[labels[i]] =colors[i]
    
    legends = ""     
    for label, color in color_label.items():
        legends += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='legend' class='legend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legends}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var Layout = (function() {{
                    var flag = false;
                    return function() {{
                        if (!flag) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!flag)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          flag = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        Layout()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .legend {
        z-index:9999;
        float:right;
        background-color: rgb(255, 255, 255);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .legend a {
        color: #777;
        }
    </style>
    """

    maps.get_root().header.add_child(folium.Element(script + css))

    return maps

In [15]:
#Plotting high-level flood_rating data

florida_flood=folium.Map(location=[lat,lon],zoom_start=9)

for d in df_vichara_loanbook.iterrows():
    html = popup_html_flood(d[0])
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        #popup ='property'
    colour_point = FloodRatingSwitch(d[1]['floodscore_ud'], d[1]['floodscore_def'])
    folium.CircleMarker(
        [d[1]['latitude'], d[1]['longitude']],
        radius=6,
        color=colours_flood[colour_point],
        fill=True,
        fill_color=colours_flood[colour_point],
        fill_opacity=0.7,
        popup=popup
    ).add_to(florida_flood)
        
florida_flood = add_legend(florida_flood, 'Likelihood of flooding', colors = list(colours_flood.values()), labels = ('Very Low', 'Low', 'Moderate', 'Moderate to High', 'High', 'Very High', 'ERROR'))

loc = ' Geo-Location Map - Floodability - Vichara Mortgages '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
florida_flood = florida_flood.get_root().html.add_child(folium.Element(title_html))
florida_flood.save('data/vichara/output/Visualization_Florida_Airbus_Vichara_loanbook/Florida_Airbus_Vichara_Flood_Rating.html') #381 MB

In [16]:
#Plotting detailed flood_rating and loan data along with index search

for key in dict_vichara_df:
    florida_flood=folium.Map(location=[lat,lon],zoom_start=9)
    geo_json = {
        "type": "FeatureCollection",
        "features": [],
    }
    for d in dict_vichara_df[key].iterrows():
        temp_dict = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates":[d[1]["longitude"], d[1]["latitude"]]    
            },
            "properties": {
                "name": d[1]["id"]
            }
        }
        geo_json["features"].append(temp_dict)
        html = popup_html_flood_loan(d[0], dict_vichara_df[key])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        #popup ='property'
        colour_point = FloodRatingSwitch(d[1]['floodscore_ud'], d[1]['floodscore_def'])
        folium.CircleMarker(
                    [d[1]['latitude'], d[1]['longitude']],
                    radius=6,
                    color=colours_flood[colour_point],
                    fill=True,
                    fill_color=colours_flood[colour_point],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(florida_flood)
    geojson_obj = folium.GeoJson(geo_json, show=False).add_to(florida_flood)
    florida_flood = add_legend(florida_flood, 'Likelihood of flooding', colors = list(colours_flood.values()), labels = ('Very Low', 'Low', 'Moderate', 'Moderate to High', 'High', 'Very High', 'ERROR'))
    Search(geojson_obj,search_label='name',placeholder='Search for ID').add_to(florida_flood)  
    folium.LayerControl().add_to(florida_flood)
    loc = ' Geo-Location Map - Floodability - Vichara Mortgages - Zip - '+key
    title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
    florida_flood = florida_flood.get_root().html.add_child(folium.Element(title_html))
    florida_flood.save('data/vichara/output/Visualization_Florida_Airbus_Vichara_loanbook/Florida_Airbus_Vichara_Flood_Rating_{}.html'.format(key)) #658 MB

In [17]:
## HeatMap Example
#### Follow this example of how to create a HEATMAP : https://wellsr.com/python/plotting-geographical-heatmaps-with-python-folium-module/ . 

from folium.plugins import HeatMap
import numpy

florida_cluster = folium.Map(location=[27.950575,-82.457176],zoom_start=19)

# HeatMap array of values
array_heatmap = []
for d in df_vichara_loanbook.iterrows(): 
         score_all_weight_0_100 = FloodRating_Ration_0_100_Value(d[1]['floodscore_ud'], d[1]['floodscore_def'])
         array_val = [[ d[1]['latitude'], d[1]['longitude'], score_all_weight_0_100 ]]
         array_heatmap = array_heatmap + array_val
# print(array_heatmap)
html = popup_html_flood(d[0])
popup = folium.Popup(folium.Html(html, script=True), max_width=500)
HeatMap(array_heatmap).add_to(florida_cluster)

loc = ' HeatMap - Floodability Risk - Vichara Mortgages '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
florida_cluster = florida_cluster.get_root().html.add_child(folium.Element(title_html))
florida_cluster
florida_cluster.save('data/vichara/output/Visualization_Florida_Airbus_Vichara_loanbook/Florida_Airbus_Vichara_Flood_Rating_heatmap.html') #4.34 MB

In [18]:
exit()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
